In [259]:
!python --version

Python 3.12.4


In [260]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings("ignore")

from sklearn.cluster import Birch
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture
from sklearn.cluster import SpectralClustering, AgglomerativeClustering, KMeans
from kmodes.kprototypes import KPrototypes

from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

#from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import cross_val_score ,StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, RobustScaler

le = LabelEncoder()
skf10 = StratifiedKFold(n_splits=10, shuffle=True,random_state=24)
skf5 = StratifiedKFold(n_splits=5, shuffle=True,random_state=24)

from lofo import LOFOImportance, Dataset, plot_importance
from sklearn.feature_selection import RFECV
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

import optuna
from scipy.stats import mode
from itertools import combinations
from tqdm.notebook import tqdm
tqdm.pandas()


# 1. Veriye İlk Bakış

In [261]:
test       = pd.read_csv('data\\test_x.csv').drop(['id'],axis=1)
train      = pd.read_csv('data\\train.csv').drop(['id'],axis=1)
train_copy = pd.read_csv('data\\train.csv').drop(['id'],axis=1)

target_col = 'Degerlendirme Puani'
categoric_cols = [col for col in train.columns if train[col].dtype == 'O']
numeric_cols = [col for col in train.columns if train[col].dtype != 'O']
numeric_cols.remove(target_col)

In [262]:
train

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,Bölüm,Universite Kacinci Sinif,Universite Not Ortalamasi,Daha Once Baska Bir Universiteden Mezun Olmus,Lise Adi,Lise Adi Diger,Lise Sehir,Lise Turu,Lise Bolumu,Lise Bolum Diger,Lise Mezuniyet Notu,Baska Bir Kurumdan Burs Aliyor mu?,Burs Aldigi Baska Kurum,Baska Kurumdan Aldigi Burs Miktari,Anne Egitim Durumu,Anne Calisma Durumu,Anne Sektor,Baba Egitim Durumu,Baba Calisma Durumu,Baba Sektor,Kardes Sayisi,Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?,Uye Oldugunuz Kulubun Ismi,Profesyonel Bir Spor Daliyla Mesgul musunuz?,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite"
0,2014,52.0,Erkek,4/6/1994,"Altindag, Ankara",Ankara,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,Bilgisayar Muhendisligi,3,3.50-3,Hayır,Ankara Ataturk Anadolu Lisesi,NaN,Ankara,Anadolu lisesi,MF,NaN,3.50-3,Hayır,NaN,NaN,Üniversite,Hayır,0,Üniversite,Emekli,0,1,Evet,Bilkent IEEE Bilgisayar Toplulugu 2013-2014 Ba...,Evet,Diğer,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN
1,2014,30.0,Erkek,6/11/1993,Üsküdar,İstanbul,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,Elektrik Elektronik Mühendisliği,3,3.00-2.50,Hayır,Betül Can Anadolu Lisesi,NaN,Ankara,Anadolu lisesi,MF,NaN,3.00 - 4.00,Hayır,NaN,NaN,Üniversite,Evet,Kamu,Yüksek Lisans / Doktora,Emekli,0,1,Hayır,NaN,Hayır,0,Hayır,NaN,Evet,Evet,Bilkent Cyberparkta bir şirkette Türkiye nin i...,NaN,NaN,NaN
2,2014,18.0,Erkek,1/15/1986,Samsun,İstanbul,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,Finans Yönetimi ve Pazarlama,1,NaN,Hayır,Batem,NaN,Ankara,Meslek lisesi,Elektirk,NaN,3.50-3,Hayır,NaN,NaN,İlkokul Mezunu,Hayır,0,İlkokul Mezunu,Evet,Özel Sektör,6,Evet,Tema,Hayır,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN
3,2014,40.0,Erkek,6/4/1991,Diyarbakır,İstanbul,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,Bilgisayar Mühendisliği,3,3.00-2.50,Hayır,Cumhuriyet Fen Lisesi,NaN,Diyarbakır,Fen lisesi,Sayısal,NaN,3.50-3,Hayır,NaN,NaN,İlkokul Mezunu,Hayır,0,Üniversite,Emekli,0,10,Evet,Güzel Konuşma ve yazma kulübü,Hayır,0,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN
4,2014,24.0,Erkek,2 Kasim 1992,Ankara/Altındağ,Ankara,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,Siyaset Bilimi ve Uluslararası ilişkiler,NaN,3.00-2.50,Hayır,Samanyolu Lisesi,NaN,Ankara,Özel lisesi,TM,NaN,3.00 - 4.00,Hayır,NaN,NaN,İlkokul Mezunu,Hayır,0,Üniversite,Emekli,0,1,Hayır,NaN,Hayır,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,Erkek,18.06.2003 00:00,İstanbul,İstanbul,İstanbul Aydın Üniversitesi,Özel,NaN,Evet,Muhasebe,2,0 - 1.79,NaN,BAĞCILAR EDİP İPLİK MESLEKİ VE TEKNİK ANADOLU ...,NaN,İstanbul,Devlet,Eşit Ağırlık,NaN,50 - 74,Hayır,-,-,İlkokul,Hayır,-,Ortaokul,Evet,Özel Sektör,4.0,Hayır,NaN,Hayır,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN
65121,2022,0.0,Erkek,18.04.2002 00:00,Hatay,Muğla,Muğla Sıtkı Koçman Üniversitesi,Devlet,NaN,Hayır,Turizm İşletmeciliği,1,Hazırlığım,NaN,Ahmet Vefik paşa Anadolu lisesi,NaN,Bursa,Devlet,Eşit Ağırlık,NaN,75 - 100,Hayır,-,-,Lise,Hayır,-,Lise,Evet,Özel Sektör,4.0,Hayır,NaN,Hayır,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN
65122,2022,0.0,Erkek,23.10.1999 00:00,Mersin,Mersin,Süleyman Demirel Üniversitesi,Devlet,NaN,Hayır,Otomotiv Mühendisliği,4,0 - 1.79,NaN,Açık lise,NaN,Mersin,Devlet,Sayısal,NaN,75 - 100,Evet,Kyk,500₺ - 999₺,Lise,Hayır,-,Lise,Hayır,-,3.0,Hayır,NaN,Hayır,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN
65123,2022,0.0,Erkek,17.11.2000 00:00,Samsun,Samsun,Karabük Üniversitesi,Devlet,NaN,Hayır,Sanat Tarihi,3,0 - 1.79,NaN,Samsun Terme Karadeniz mesleki teknik Anadolu ...

In [263]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65125 entries, 0 to 65124
Data columns (total 43 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Basvuru Yili                                               65125 non-null  int64  
 1   Degerlendirme Puani                                        65124 non-null  float64
 2   Cinsiyet                                                   64956 non-null  object 
 3   Dogum Tarihi                                               64948 non-null  object 
 4   Dogum Yeri                                                 64334 non-null  object 
 5   Ikametgah Sehri                                            63088 non-null  object 
 6   Universite Adi                                             64993 non-null  object 
 7   Universite Turu                                            64870 non-null  object 
 8   Burslu

In [264]:
train[cat_cols].describe().T

,count,unique,top,freq
Cinsiyet,64956,4,Kadın,32077
Dogum Tarihi,64948,13498,1/1/70 2:00,1655
Dogum Yeri,64334,2230,İstanbul,8682
Ikametgah Sehri,63088,319,İstanbul,16625
Universite Adi,64993,527,İSTANBUL ÜNİVERSİTESİ,2083
Universite Turu,64870,4,Devlet,37885
Burs Aliyor mu?,65125,5,Hayır,51978
Bölüm,64894,7748,Endüstri Mühendisliği,2434
Universite Kacinci Sinif,64751,12,2,21773
Universite Not Ortalamasi,62372,20,3.00-2.50,10758


In [265]:
train[num_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
Basvuru Yili,65125.0,2018.277298,2.669979,2014.0,2016.0,2019.0,2021.0,2022.0
Degerlendirme Puani,65124.0,32.086466,18.139239,0.0,18.0,29.0,44.0,102.0
Burslu ise Burs Yuzdesi,22440.0,34.226181,41.632412,0.0,0.0,0.0,75.0,100.0


# 2. Veriyi Temizleme

## 2.1 Basvuru Yili

In [266]:
train[["Basvuru Yili"]] = train[["Basvuru Yili"]].astype(str)

In [267]:
train["Basvuru Yili"].dtype

#Label Encoder kullanırız sonra

dtype('O')

## 2.2 Cinsiyet

In [268]:
train['Cinsiyet'].value_counts()

Cinsiyet
Kadın                    32077
Erkek                    22752
ERKEK                    10012
Belirtmek istemiyorum      115
Name: count, dtype: int64

In [269]:
train['Cinsiyet'] = train['Cinsiyet'].str.capitalize()
train['Cinsiyet'].value_counts()

Cinsiyet
Erkek                    32764
Kadın                    32077
Belirtmek istemiyorum      115
Name: count, dtype: int64

In [270]:
# One-hot encoding
train = pd.get_dummies(train, columns=['Cinsiyet'])
train

,Basvuru Yili,Degerlendirme Puani,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,Bölüm,Universite Kacinci Sinif,Universite Not Ortalamasi,Daha Once Baska Bir Universiteden Mezun Olmus,Lise Adi,Lise Adi Diger,Lise Sehir,Lise Turu,Lise Bolumu,Lise Bolum Diger,Lise Mezuniyet Notu,Baska Bir Kurumdan Burs Aliyor mu?,Burs Aldigi Baska Kurum,Baska Kurumdan Aldigi Burs Miktari,Anne Egitim Durumu,Anne Calisma Durumu,Anne Sektor,Baba Egitim Durumu,Baba Calisma Durumu,Baba Sektor,Kardes Sayisi,Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?,Uye Oldugunuz Kulubun Ismi,Profesyonel Bir Spor Daliyla Mesgul musunuz?,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",Cinsiyet_Belirtmek istemiyorum,Cinsiyet_Erkek,Cinsiyet_Kadın
0,2014,52.0,4/6/1994,"Altindag, Ankara",Ankara,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,Bilgisayar Muhendisligi,3,3.50-3,Hayır,Ankara Ataturk Anadolu Lisesi,NaN,Ankara,Anadolu lisesi,MF,NaN,3.50-3,Hayır,NaN,NaN,Üniversite,Hayır,0,Üniversite,Emekli,0,1,Evet,Bilkent IEEE Bilgisayar Toplulugu 2013-2014 Ba...,Evet,Diğer,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,False,True,False
1,2014,30.0,6/11/1993,Üsküdar,İstanbul,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,Elektrik Elektronik Mühendisliği,3,3.00-2.50,Hayır,Betül Can Anadolu Lisesi,NaN,Ankara,Anadolu lisesi,MF,NaN,3.00 - 4.00,Hayır,NaN,NaN,Üniversite,Evet,Kamu,Yüksek Lisans / Doktora,Emekli,0,1,Hayır,NaN,Hayır,0,Hayır,NaN,Evet,Evet,Bilkent Cyberparkta bir şirkette Türkiye nin i...,NaN,NaN,NaN,False,True,False
2,2014,18.0,1/15/1986,Samsun,İstanbul,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,Finans Yönetimi ve Pazarlama,1,NaN,Hayır,Batem,NaN,Ankara,Meslek lisesi,Elektirk,NaN,3.50-3,Hayır,NaN,NaN,İlkokul Mezunu,Hayır,0,İlkokul Mezunu,Evet,Özel Sektör,6,Evet,Tema,Hayır,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,False,True,False
3,2014,40.0,6/4/1991,Diyarbakır,İstanbul,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,Bilgisayar Mühendisliği,3,3.00-2.50,Hayır,Cumhuriyet Fen Lisesi,NaN,Diyarbakır,Fen lisesi,Sayısal,NaN,3.50-3,Hayır,NaN,NaN,İlkokul Mezunu,Hayır,0,Üniversite,Emekli,0,10,Evet,Güzel Konuşma ve yazma kulübü,Hayır,0,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,False,True,False
4,2014,24.0,2 Kasim 1992,Ankara/Altındağ,Ankara,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,Siyaset Bilimi ve Uluslararası ilişkiler,NaN,3.00-2.50,Hayır,Samanyolu Lisesi,NaN,Ankara,Özel lisesi,TM,NaN,3.00 - 4.00,Hayır,NaN,NaN,İlkokul Mezunu,Hayır,0,Üniversite,Emekli,0,1,Hayır,NaN,Hayır,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,18.06.2003 00:00,İstanbul,İstanbul,İstanbul Aydın Üniversitesi,Özel,NaN,Evet,Muhasebe,2,0 - 1.79,NaN,BAĞCILAR EDİP İPLİK MESLEKİ VE TEKNİK ANADOLU ...,NaN,İstanbul,Devlet,Eşit Ağırlık,NaN,50 - 74,Hayır,-,-,İlkokul,Hayır,-,Ortaokul,Evet,Özel Sektör,4.0,Hayır,NaN,Hayır,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,False,True,False
65121,2022,0.0,18.04.2002 00:00,Hatay,Muğla,Muğla Sıtkı Koçman Üniversitesi,Devlet,NaN,Hayır,Turizm İşletmeciliği,1,Hazırlığım,NaN,Ahmet Vefik paşa Anadolu lisesi,NaN,Bursa,Devlet,Eşit Ağırlık,NaN,75 - 100,Hayır,-,-,Lise,Hayır,-,Lise,Evet,Özel Sektör,4.0,Hayır,NaN,Hayır,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,False,True,False
65122,2022,0.0,23.10.1999 00:00,Mersin,Mersin,Süleyman Demirel Üniversitesi,Devlet,NaN,Hayır,Otomotiv Mühendisliği,4,0 - 1.79,NaN,Açık lise,NaN,Mersin,Devlet,Sayısal,NaN,75 - 100,Evet,Kyk,500₺ - 999₺,Lise,Hayır,-,Lise,Hayır,-,3.0,Hayır,NaN,Hayır,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,False,True,False
65123,2022,0.0,17.1

## 2.3 Dogum Tarihi - Dogum Yili 

In [271]:
# Mapping of Turkish month names to English
turkish_to_english_months = {
    "Ocak": "January",
    "Şubat": "February",
    "Subat": "February",
    "Mart": "March",
    "Nisan": "April",
    "Mayıs": "May",
    "Mayis": "May",
    "Haziran": "June",
    "Temmuz": "July",
    "Ağustos": "August",
    "Agustos": "August",
    "Eylül": "September",
    "Eylul": "September",
    "Ekim": "October",
    "Kasım": "November",
    "Kasim": "November",
    "Aralık": "December",
    "Aralik": "December"
}

def normalize_turkish_months(date_str):
    for turkish_month, english_month in turkish_to_english_months.items():
        date_str = re.sub(r'\b' + turkish_month + r'\b', english_month, date_str)
    return date_str

def extract_year(date_str):
    if not isinstance(date_str, str):
        return None  # Return None if the input is not a string
    try:
        # Normalize the date string for Turkish month names
        date_str = normalize_turkish_months(date_str)
        
        # Parse the date string using dateutil.parser
        parsed_date = parser.parse(date_str, fuzzy=True, dayfirst=False)
        
        # Extract the year from the parsed date
        return parsed_date.year
    except (ValueError, TypeError):
        # If parsing fails, try extracting the year directly from the string
        year_match = re.search(r'(\d{4})$', date_str)
        if year_match:
            return int(year_match.group(1))
        return None
    


train['Dogum Yili'] = train['Dogum Tarihi'].apply(extract_year)
train['Dogum Yili'].isnull().sum()

192

In [272]:
train['Dogum Yili'].value_counts()

Dogum Yili
2000.0    7189
1999.0    6690
1998.0    6623
1995.0    6301
1996.0    6158
1997.0    6122
2001.0    5675
1994.0    5398
1993.0    4165
2002.0    3212
1992.0    2585
2070.0    1661
1991.0    1397
2003.0     737
1990.0     601
1989.0      81
1970.0      55
1988.0      49
1987.0      32
2016.0      24
2004.0      23
1986.0      21
2017.0      11
2005.0       9
2020.0       9
199.0        8
1984.0       7
1985.0       6
1197.0       6
200.0        5
1976.0       5
1983.0       5
1980.0       4
1196.0       4
1977.0       3
197.0        3
1979.0       2
996.0        2
9691.0       2
9951.0       2
998.0        2
1199.0       2
2011.0       2
993.0        2
1195.0       2
2015.0       1
7199.0       1
328.0        1
2006.0       1
1204.0       1
808.0        1
2009.0       1
198.0        1
989.0        1
9941.0       1
6123.0       1
1929.0       1
1198.0       1
9590.0       1
1155.0       1
5778.0       1
1194.0       1
2996.0       1
1896.0       1
9916.0       1
1888.0       1

In [273]:
train['Dogum Yili'] = train['Dogum Yili'].apply(lambda x: np.nan if (x > 2005) | (x < 1980) else x)
train['Dogum Yili'].isnull().sum()

2033

In [274]:
train['Dogum Yili'].value_counts()

Dogum Yili
2000.0    7189
1999.0    6690
1998.0    6623
1995.0    6301
1996.0    6158
1997.0    6122
2001.0    5675
1994.0    5398
1993.0    4165
2002.0    3212
1992.0    2585
1991.0    1397
2003.0     737
1990.0     601
1989.0      81
1988.0      49
1987.0      32
2004.0      23
1986.0      21
2005.0       9
1984.0       7
1985.0       6
1983.0       5
1980.0       4
1982.0       1
1981.0       1
Name: count, dtype: int64

## 2.4 Basvuru Yasi

In [275]:
train['Basvuru Yasi'] = train['Basvuru Yili'].astype(float) - train['Dogum Yili']
train['Basvuru Yasi'].value_counts()

Basvuru Yasi
21.0    15764
20.0    15046
22.0    12487
19.0     8085
23.0     7134
24.0     3073
25.0      781
18.0      394
26.0      117
17.0       58
27.0       56
28.0       35
30.0       20
29.0       13
31.0        8
34.0        7
32.0        3
33.0        3
16.0        2
35.0        2
14.0        1
11.0        1
36.0        1
40.0        1
Name: count, dtype: int64

In [276]:
train['Basvuru Yasi'].isnull().sum()

2033

## 2.5 Dogum Yeri

In [ ]:
train["Dogum Yeri"].value_counts().head(81).sum()

In [285]:
def standardize_general(location):

    if 'istanbul'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'İstanbul'
    if 'i̇stanbul'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'İstanbul'
    elif 'ıstanbul' in location.lower():
        return 'İstanbul'
    
    elif 'ankara'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Ankara'
    elif 'altındag'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Ankara'
    elif 'altindag'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Ankara'
    elif 'altındağ'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Ankara'
    elif 'altindağ'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Ankara'
    
    elif 'izmir'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'İzmir'
    elif 'ızmir'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'İzmir'
    
    elif 'adana'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Adana'
    
    elif 'bursa'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Bursa'
    elif 'konya'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Konya'
    elif 'mersin'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Mersin'
    elif 'trabzon'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Trabzon'
    elif 'kocaeli'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Kocaeli'
    
    elif 'k.maraş'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Kahramanmaraş'
    
    else:
        return location.capitalize()

train["Dogum Yeri"] = train["Dogum Yeri"].astype(str).apply(standardize_general)
train["Dogum Yeri"].value_counts().head(50)

Dogum Yeri
İstanbul          11073
Ankara             4309
İzmir              2767
Bursa              1931
Adana              1914
Konya              1514
Mersin             1472
Diyarbakır         1403
Hatay              1333
Gaziantep          1330
Antalya            1199
Kayseri            1152
Van                1122
Şanlıurfa          1101
Kocaeli             995
Mardin              990
Malatya             963
Erzurum             930
Samsun              929
Manisa              860
Nan                 791
Kahramanmaraş       762
Denizli             725
Trabzon             649
Sivas               640
Batman              619
Ağrı                605
Balıkesir           599
Ordu                561
Eskişehir           555
Adıyaman            534
Sakarya             520
Aydın               516
Tokat               509
Çorum               494
Zonguldak           448
Yozgat              433
Elazığ              430
Diğer               405
Şırnak              393
Tekirdağ            392
Muş  

In [ ]:
train["Dogum Yeri"].value_counts().head